In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import max_norm

In [37]:
def calculate_moving_averages(df, window_sizes):
    for window_size in window_sizes:
        column_name = f'MA_{window_size}'
        df[column_name] = df['close'].rolling(window=window_size).mean()
    return df

In [38]:
df = pd.read_csv('bnb_historical_data.csv')

In [39]:
window_sizes = [7, 25, 99]
df = calculate_moving_averages(df, window_sizes)

In [47]:
features = df[['MA_7', 'MA_25', 'MA_99', 'open', 'high', 'low', 'close', 'volume']].dropna().values
target = df['close'].values.reshape(-1, 1)

# Normalize features using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler.fit_transform(features)

In [48]:
print(features)

[[245.62857143 245.78       246.01313131 ... 245.5        245.6
   87.649     ]
 [245.6        245.768      246.00606061 ... 245.5        245.6
   55.016     ]
 [245.61428571 245.76       246.         ... 245.5        245.7
   37.494     ]
 ...
 [310.31428571 310.428      309.16767677 ... 310.3        310.5
  255.438     ]
 [310.3        310.424      309.19292929 ... 310.4        310.5
  618.347     ]
 [310.38571429 310.408      309.21919192 ... 310.4        310.6
  528.993     ]]


In [49]:
def create_sequences(data, target, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        label = target[i + sequence_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [50]:
sequence_length = 10  # Adjust as needed
X, y = create_sequences(features_scaled, target, sequence_length)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]),
               kernel_constraint=max_norm(3)))
model.add(Dense(units=1))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

In [54]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/5
11833/11833 [==============================] - 25s 2ms/step - loss: 2194.0562 - val_loss: 1.4362
Epoch 2/5
11833/11833 [==============================] - 24s 2ms/step - loss: 1.4136 - val_loss: 1.2446
Epoch 3/5
11833/11833 [==============================] - 24s 2ms/step - loss: 0.9866 - val_loss: 0.6993
Epoch 4/5
11833/11833 [==============================] - 24s 2ms/step - loss: 0.8616 - val_loss: 0.7057
Epoch 5/5
11833/11833 [==============================] - 24s 2ms/step - loss: 0.7986 - val_loss: 0.9225


In [55]:
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

3287/3287 [==============================] - 3s 990us/step - loss: 0.9309
Mean Squared Error on Test Set: 0.9309398531913757


In [56]:
import pickle

In [57]:
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)